In [ ]:
import os
import sys
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import logging
from typing import Dict, Any, List

# --- 프로젝트 루트 경로 설정 ---
try:
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
    print(f"Added project root to path: {PROJECT_ROOT}")

# --- src 모듈 임포트 ---
try:
    from src.utils import load_config, load_jsonl
    # 평가 함수 임포트
    from src.evaluation import calculate_qa_metrics_for_item # PreciseWikiQA용
    # from src.evaluation import evaluate_factscore_official # FactScore용 (구현 필요)
    # from src.evaluation import evaluate_truthfulqa_official # TruthfulQA용 (구현 필요)
except ImportError as e:
    print(f"Error importing src modules: {e}")

# 로깅 및 스타일 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(module)s - %(message)s')
logger = logging.getLogger(__name__)
sns.set_theme(style="whitegrid", context="talk")
pd.set_option('display.max_rows', 100)

print("Libraries and modules imported successfully.")

In [ ]:
# --- 1. 설정 로드 ---
CONFIG_PATH = os.path.join(PROJECT_ROOT, "configs", "config.yaml")
config = load_config(CONFIG_PATH)
RESULTS_BASE_DIR = os.path.join(PROJECT_ROOT, config.get('results_base_dir', 'results'))
print(f"Loading all results from: {RESULTS_BASE_DIR}")

# --- 2. 평가 함수 매핑 (플레이스홀더 포함) ---
def evaluate_factscore_placeholder(results_data, config):
    """FactScore 공식 스크립트 연동 전 임시 함수"""
    logger.warning("Using PLACEHOLDER for FactScore evaluation!")
    # TODO: 'final_output'과 'query'/'topic'을 FactScore 공식 스크립트에 전달
    # 예시: return factscore.get_score(...)
    # 임시로 랜덤 점수 반환 (분석용)
    import random
    return {'factscore': random.uniform(0.5, 0.8)} 

def evaluate_truthfulqa_placeholder(results_data, config):
    """TruthfulQA 공식 스크립트 연동 전 임시 함수"""
    logger.warning("Using PLACEHOLDER for TruthfulQA evaluation!")
    # TODO: 'final_output', 'correct_answers_truthfulqa' 등을 공식 스크립트에 전달
    # 예시: return truthfulqa.evaluate(...)
    # 임시로 랜덤 점수 반환 (분석용)
    import random
    return {'accuracy': random.uniform(0.4, 0.7)}

def evaluate_qa_benchmark(results_data: List[Dict[str, Any]], config: Dict[str, Any]) -> Dict[str, Any]:
    """PreciseWikiQA / MultiSpanQA 평가 (src/evaluation.py 재사용)"""
    logger.info("Running QA (EM/F1) evaluation...")
    em_sum = 0
    f1_sum = 0
    evaluated_count = 0
    for item in results_data:
        pred = item.get("method_result", {}).get("final_output")
        gold_list = item.get("answers") # data_loader가 'answers' 키로 저장
        if pred is not None and gold_list is not None:
            metrics = calculate_qa_metrics_for_item(pred, gold_list)
            em_sum += metrics['em']
            f1_sum += metrics['f1']
            evaluated_count += 1
    em_avg = (em_sum / evaluated_count) * 100 if evaluated_count > 0 else 0
    f1_avg = (f1_sum / evaluated_count) * 100 if evaluated_count > 0 else 0
    return {"accuracy_em": em_avg, "f1_score": f1_avg, "count": evaluated_count}

# 데이터셋 이름(키)과 실제 평가 함수 매핑
EVALUATION_MAP = {
    'hallulens_longwiki': evaluate_factscore_placeholder, # TODO: 공식 FactScore 함수로 교체
    'hallulens_precisewikiqa': evaluate_qa_benchmark,
    'truthfulqa': evaluate_truthfulqa_placeholder, # TODO: 공식 TruthfulQA 함수로 교체
    # 'multispanqa_dev': evaluate_qa_benchmark, # MultiSpanQA 사용 시
}

# --- 3. 모든 결과 파일 로드, 평가, 집계 ---
all_results_data = []
# `results/` 하위의 모든 .jsonl 파일 탐색
file_paths = glob.glob(os.path.join(RESULTS_BASE_DIR, "**", "*.jsonl"), recursive=True)

if not file_paths:
    logger.error(f"[오류] {RESULTS_BASE_DIR}에서 결과 파일을 찾을 수 없습니다. 실험을 먼저 실행하세요.")
else:
    print(f"Found {len(file_paths)} result files to analyze.")

# 파일명에서 정보 파싱을 위한 정규 표현식 (예시)
# serc_t2_mf5_... -> method=serc, t_max=2, max_facts=5
# serc_dense_iterative_t2... -> method=serc_dense, t_max=2
# baseline... -> method=baseline
# cove... -> method=cove
# serc_t1_mf5... -> method=serc_1pass (t=1일 때)
serc_regex = re.compile(r"serc_t(\d+)_mf(\d+)")
dense_regex = re.compile(r"serc_dense_iterative_t(\d+)")

for f_path in file_paths:
    try:
        filename = os.path.basename(f_path)
        parts = f_path.replace("\\", "/").split('/')
        dataset_name = parts[-2]
        model_name = parts[-3].replace('_', '/') # Qwen_Qwen... -> Qwen/Qwen...
        
        # 파일명 기반으로 방법론(method) 및 파라미터 파싱
        method = "unknown"
        t_max = None
        max_facts = None
        
        if filename.startswith("baseline"):
            method = "Baseline"
        elif filename.startswith("cove"):
            method = "CoVe"
        elif "dense" in filename:
            method = "SERC (Dense)"
            match_dense = dense_regex.search(filename)
            if match_dense: t_max = int(match_dense.group(1))
        elif filename.startswith("serc"):
            match_serc = serc_regex.search(filename)
            if match_serc:
                t_max = int(match_serc.group(1))
                max_facts = int(match_serc.group(2))
                method = "SERC (1-pass)" if t_max == 1 else "SERC (Iterative)"
            else:
                method = "SERC (Unknown)"
        
        if method == "unknown":
            logger.warning(f"Skipping file (method unknown): {filename}")
            continue
            
        if dataset_name not in EVALUATION_MAP:
            logger.warning(f"Skipping file (no evaluator for dataset '{dataset_name}'): {filename}")
            continue

        # --- 4. 실제 평가 수행 ---
        logger.info(f"Evaluating: {model_name} / {dataset_name} / {method} (T={t_max}, MF={max_facts})")
        results_data = load_jsonl(f_path)
        if not results_data:
            logger.warning(f"Result file is empty: {f_path}")
            continue
            
        # 데이터셋에 맞는 평가 함수 호출
        eval_func = EVALUATION_MAP[dataset_name]
        metrics = eval_func(results_data, config)
        
        # 토큰 사용량 집계 (예시: SERC 결과에서 누적 토큰 가져오기)
        # TODO: SERC 함수가 'total_tokens_used'를 반환하도록 수정 필요
        total_tokens = 0
        if method.startswith("SERC"):
            # 예시: 각 항목의 serc_result에서 'total_tokens_used' 키 값 합산
            try:
                total_tokens = sum(item['method_result']['serc_result'].get('total_tokens_used', 0) for item in results_data)
            except KeyError:
                total_tokens = 0 # 구현 안된 경우
        
        # 집계용 딕셔너리 생성
        entry = {
            "Model": model_name,
            "Dataset": dataset_name,
            "Method": method,
            "T_max": t_max,
            "Max_Facts": max_facts,
            "Total_Tokens": total_tokens, # 총 토큰
            "Avg_Tokens": total_tokens / len(results_data) if results_data else 0,
            **metrics # 평가 함수가 반환한 모든 지표 (factscore, f1_score 등)
        }
        all_results_data.append(entry)

    except Exception as e:
        logger.error(f"Error processing file {f_path}: {e}", exc_info=True)

# --- 5. 최종 집계 데이터프레임 생성 ---
df_final_results = pd.DataFrame(all_results_data)
print("\n--- 최종 집계 결과 (Head) ---")
display(df_final_results.head())

In [ ]:
# RQ1 (SOTA) & RQ4 (Scaling)
# - 비교 대상: Baseline, CoVe, SERC (Iterative)
# - (참고) T_max, Max_Facts 값이 여러 개일 수 있으므로,
#   예비 실험(2_pilot...)에서 결정된 최적의 값 (e.g., T=2, MF=5)으로 필터링 필요

OPTIMAL_T_MAX = 2 # 예비 실험에서 결정된 값 (예시)
OPTIMAL_MAX_FACTS = 5 # 예비 실험에서 결정된 값 (예시)

# 비교 대상 필터링
methods_to_compare_rq1 = ["Baseline", "CoVe", "SERC (Iterative)"]
df_rq1_rq4 = df_final_results[
    (df_final_results['Method'].isin(methods_to_compare_rq1))
].copy()

# SERC (Iterative)는 최적 파라미터 사용 결과만 선택
df_rq1_rq4 = df_rq1_rq4[
    (df_rq1_rq4['Method'] != 'SERC (Iterative)') | 
    ((df_rq1_rq4['Method'] == 'SERC (Iterative)') & 
     (df_rq1_rq4['T_max'] == OPTIMAL_T_MAX) & 
     (df_rq1_rq4['Max_Facts'] == OPTIMAL_MAX_FACTS))
]

# (필요시 'Method' 이름 정리)
# df_rq1_rq4['Method'] = df_rq1_rq4['Method'].replace({'SERC (Iterative)': 'SERC (Ours)'})

print(f"\n--- RQ1 & RQ4 분석용 테이블 (최적 파라미터 T={OPTIMAL_T_MAX}, MF={OPTIMAL_MAX_FACTS} 기준) ---")
# 모델, 데이터셋, 방법론별 성능 지표 (예: f1_score)
metrics_to_show = ['Model', 'Dataset', 'Method', 'f1_score', 'accuracy_em', 'factscore', 'Avg_Tokens']
# df_rq1_rq4_pivot = df_rq1_rq4.pivot_table(
#     index=['Model', 'Dataset'],
#     columns='Method',
#     values=['f1_score', 'factscore', 'accuracy_em'] # 보여줄 지표
# )
# display(df_rq1_rq4_pivot)
display(df_rq1_rq4[[col for col in metrics_to_show if col in df_rq1_rq4.columns]].sort_values(by=['Model', 'Dataset']))


# --- RQ4 시각화 (예시: 8B 모델 대비 30B 모델 성능) ---
# (데이터셋별로 그래프 그리기)
for dataset in df_rq1_rq4['Dataset'].unique():
    plt.figure(figsize=(10, 6))
    sns.barplot(
        data=df_rq1_rq4[df_rq1_rq4['Dataset'] == dataset],
        x='Model',
        y='f1_score', # 또는 'factscore'
        hue='Method'
    )
    plt.title(f"RQ1 & RQ4: {dataset} 성능 비교")
    plt.ylabel("Performance (F1 Score or FactScore)")
    plt.xlabel("Model")
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.xticks(rotation=15)
    plt.show()

In [ ]:
# RQ2 (Low-Density vs Dense)
# - 비교 대상: SERC (Iterative) vs SERC (Dense)
# - 데이터셋: HalluLens (LongWiki)
# - T_max는 동일한 값 (e.g., 최적 T_max)으로 필터링

OPTIMAL_T_MAX = 2 # 예비 실험에서 결정된 값

methods_to_compare_rq2 = ["SERC (Iterative)", "SERC (Dense)"]
df_rq2 = df_final_results[
    (df_final_results['Method'].isin(methods_to_compare_rq2)) &
    (df_final_results['Dataset'] == 'hallulens_longwiki') & # LongWiki 벤치마크
    (df_final_results['T_max'] == OPTIMAL_T_MAX) # 동일 T_max
].copy()

# SERC (Iterative)는 최적 Max_Facts 사용
df_rq2 = df_rq2[
    (df_rq2['Method'] != 'SERC (Iterative)') |
    (df_rq2['Max_Facts'] == OPTIMAL_MAX_FACTS)
]
# (SERC (Dense)는 max_facts가 없을 수 있으므로 N/A 처리 필요)

print(f"\n--- RQ2 분석용 테이블 (LongWiki, T_max={OPTIMAL_T_MAX} 기준) ---")
metrics_to_show_rq2 = ['Model', 'Method', 'factscore', 'Avg_Tokens']
display(df_rq2[[col for col in metrics_to_show_rq2 if col in df_rq2.columns]].sort_values(by=['Model']))

# --- RQ2 시각화 (예: FactScore vs Avg_Tokens) ---
plt.figure(figsize=(10, 6))
sns.scatterplot(
    data=df_rq2,
    x='Avg_Tokens',
    y='factscore',
    hue='Model',
    style='Method',
    s=200 # 마커 크기
)
plt.title(f"RQ2: 효율성 vs 정확성 (LongWiki, T_max={OPTIMAL_T_MAX})")
plt.xlabel("Average Tokens Used (Efficiency)")
plt.ylabel("FactScore (Accuracy)")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
# RQ3 (Iterative vs 1-pass)
# - 비교 대상: SERC (Iterative) vs SERC (1-pass)
# - 데이터셋: HalluLens (LongWiki)
# - Max_Facts는 동일한 값 (e.g., 최적 max_facts)으로 필터링

OPTIMAL_MAX_FACTS = 5 # 예비 실험에서 결정된 값

methods_to_compare_rq3 = ["SERC (Iterative)", "SERC (1-pass)"]
df_rq3 = df_final_results[
    (df_final_results['Method'].isin(methods_to_compare_rq3)) &
    (df_final_results['Dataset'] == 'hallulens_longwiki') & # LongWiki 벤치마크
    (df_final_results['Max_Facts'] == OPTIMAL_MAX_FACTS) # 동일 Max_Facts
].copy()

# (SERC (Iterative)는 T_max=OPTIMAL_T_MAX, SERC (1-pass)는 T_max=1 이어야 함)
df_rq3 = df_rq3[
    ((df_rq3['Method'] == 'SERC (1-pass)') & (df_rq3['T_max'] == 1)) |
    ((df_rq3['Method'] == 'SERC (Iterative)') & (df_rq3['T_max'] == OPTIMAL_T_MAX))
]

print(f"\n--- RQ3 분석용 테이블 (LongWiki, Max_Facts={OPTIMAL_MAX_FACTS} 기준) ---")
metrics_to_show_rq3 = ['Model', 'Method', 'T_max', 'factscore']
display(df_rq3[[col for col in metrics_to_show_rq3 if col in df_rq3.columns]].sort_values(by=['Model']))

# --- RQ3 시각화 ---
plt.figure(figsize=(10, 6))
sns.barplot(
    data=df_rq3,
    x='Model',
    y='factscore',
    hue='Method'
)
plt.title(f"RQ3: 반복(Iterative) vs 1-Pass 효과 (LongWiki, Max_Facts={OPTIMAL_MAX_FACTS})")
plt.ylabel("FactScore")
plt.xlabel("Model")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xticks(rotation=15)
plt.show()